# Car Sequencing problem in CPMPy, with visualization

Given different types of cars (based on the present options), an overall car demand per type 
and some constraints on how many times properties can be scheduled in consecutive timeslots,
the program finds a feasible sequencing of the different cars for a timetable.

This problem is also described in CSPlib: https://www.csplib.org/Problems/prob001/

Implemenation based on Alexander Schiendorfer's, but with simplified visualization
https://github.com/Alexander-Schiendorfer/cp-examples/tree/main/car-sequencing

This example requires you to install _pandas_ and _matplotlib_.

In [ ]:
! pip install pandas matplotlib --quiet

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from cpmpy import *

In [14]:
# the CPMpy model and variables
def model_car_sequence(demand, per_slots, at_most, requires):
    n_cars = sum(demand)  # The amount of cars to sequence
    n_options = len(at_most)  # The amount of different options
    n_types = len(demand)  # The amount of different car types
    requires = cpm_array(requires)  # For element constraint

    # Decision variables
    slots = intvar(0, n_types-1, shape=n_cars, name="slots")  # Sequence of different car types (offset 0)
    setup = boolvar(shape=(n_cars, n_options), name="setup")  # Sequence of different options based on the car type

    m = Model()

    # The amount of each type of car in the sequence has to be equal to the demand for that type
    m += [sum(slots == t) == demand[t] for t in range(n_types)]

    # Make sure that the options in the setup table correspond to those of the car type
    for s in range(n_cars):
        m += [setup[s,o] == requires[slots[s],o] for o in range(n_options)]

    # Check that no more than "at most" car options are used per "per_slots" slots
    for o in range(n_options):
        for s in range(n_cars - per_slots[o] + 1):
            slotrange = range(s, s + per_slots[o])
            m += (sum(setup[slotrange, o]) <= at_most[o])

    return m, (slots, setup)

In [4]:
# the visualisation: build dataframe, return/display it (is automatically nicely)
def visualize_car_sequence(slots, setup):
    df = pd.DataFrame(setup).astype(str).replace('True', 'X').replace('False', '')
    df['Config (slots)'] = [f"Options {l}" for l in slots]  # insert as first column
    df = df.set_index('Config (slots)')
    return df

In [8]:
# Example data for car sequencing
at_most = [1, 2, 2, 2, 1]  # The amount of times a property can be present in a group of consecutive timeslots (see next variable)
per_slots = [2, 3, 3, 5, 5]  # The amount of consecutive timeslots
demand = [1, 1, 2, 2, 2, 2]  # The demand per type of car
requires = [[1, 0, 1, 1, 0],
            [0, 0, 0, 1, 0],
            [0, 1, 0, 0, 1],
            [0, 1, 0, 1, 0],
            [1, 0, 1, 0, 0],
            [1, 1, 0, 0, 0]]  # The properties per type of car


In [15]:
# model, solve, visualize
(model, (slots, setup)) = model_car_sequence(demand, per_slots, at_most, requires)

sat = model.solve()
if not sat: raise Exception("No solution found.")
    
print("Slots:", slots.value())
#print("Setup:", setup.value())

visualize_car_sequence(slots.value(), setup.value())

Slots: [0 3 4 2 4 3 5 1 2 5]


,0,1,2,3,4
Config (slots),,,,,
Options 0,X,,X,X,
Options 3,,X,,X,
Options 4,X,,X,,
Options 2,,X,,,X
Options 4,X,,X,,
Options 3,,X,,X,
Options 5,X,X,,,
Options 1,,,,X,
Options 2,,X,,,X
